In [ ]:
from flask import Flask, render_template, redirect, url_for, request, jsonify
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import json
import pickle
import PSB_manage as pm

app = Flask(__name__)
model = pickle.load(open('data2.pkl', 'rb'))
cols_check = model.get_booster().feature_names

@app.route('/', methods=['POST'])
def home():
    jsonData = request.get_json()

    data1 = int(jsonData['fire'])
    #data2 = int(request.form['b']) #safe_center
    #data3 = float(request.form['c']) #onspot
    data4 = int(jsonData['emr'])
    data5 = int(jsonData['rel'])
    data6 = int(jsonData['stat'])
    data7 = int(jsonData['jur'])
    data8 = float(jsonData['x'])
    data9 = float(jsonData['y'])

    data2 = pm.get_safe(data8, data9)
    data3 = pm.dist_only(data8, data9)


    # df = pd.DataFrame(columns=['GOUT_FIRESTTN_NM', 'GOUT_SAFE_CENTER_NM', 'ONSPOT_DSTN',
    #    'EMRLF_EMD_NM', 'RELIF_OCCURPLC_TYPE', 'STATMNT_TM','JURISD_DIV_NM_CNT'])
    # df = df.append({'GOUT_FIRESTTN_NM': data1, 'GOUT_SAFE_CENTER_NM': data2, 'ONSPOT_DSTN': data3, 'EMRLF_EMD_NM': data4,
    #                 'RELIF_OCCURPLC_TYPE': data5, 'STATMNT_TM': data6, 'JURISD_DIV_NM_CNT': data7}, ignore_index=True)
    # df2 = df.astype(np.int64)
    # df3 = df2.ONSPOT_DSTN.astype(np.float64)
    # df2 = df2.drop(columns='ONSPOT_DSTN')
    # df4 = pd.concat((df3,df2), axis=1)
    # df4 = df4[cols_check]

    arr = np.array([[data1, data2, data3, data4, data5, data6, data7]])
    pred = model.predict(arr)

    # pred = model.predict(df4)
    answer = str(round(float(pred[0])))
    return jsonify({"answer" : answer})





if __name__ == "__main__":
    app.debug = True
    #app.run(port=80)
    app.run(host="192.168.2.147", port=80)


In [ ]:
##2021_04_14_1443

from flask import Flask, render_template, redirect, url_for, request, jsonify
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import json
import pickle
import PSB_manage as pm

app = Flask(__name__)
model = pickle.load(open('data2.pkl', 'rb')) # 모델 Input (XGBorlfhost)
center_model = pickle.load(open('data_goni.pkl', 'rb')) # 소요거리 모델
#cols_check = center_model.get_booster().feature_names

@app.route('/predict2', methods=['POST'])
def centerPredict():
    # 센터, 시간대, 소요시간, 의식상태, 관할여부 받기
    # 모델에 넣고 거리 리턴
    jsonData = request.get_json()
    data1 = str(jsonData['safe_center'])
    data2 = int(jsonData['jurisd'])
    data3 = int(jsonData['statmt_tm'])
    data4 = float(jsonData['move_tm'])
    data5 = int(jsonData['conscs'])
    # json객체 출력 확인
    # 119안전센터 label 값 가져오기

    center_label = int(pm.get_centerlabel(data1))

    #print(center_label)
    # 소요시간 예측하기###########################################
    # 예측 데이터 담을 df 생성
    df = pd.DataFrame(columns=['GOUT_SAFE_CENTER_NM', 'STATMNT_TM', 'MOVE_TM',
                               'JURISD_DIV_NM_CNT', 'CONSCS_STATE_TYPE'])
    # df에 행 추가
    df = df.append({'GOUT_SAFE_CENTER_NM': center_label, 'JURISD_DIV_NM_CNT': data2, 'STATMNT_TM': data3,
                    'MOVE_TM': data4, 'CONSCS_STATE_TYPE': data5}, ignore_index=True)
    # df 타입 변경
    df2 = df.astype(np.int64)
    df3 = df2.MOVE_TM.astype(np.float64)
    df2 = df2.drop(columns='MOVE_TM')
    df4 = pd.concat((df3, df2), axis=1)
    # 예측하기
    pred = center_model.predict(df4)

    answer = str(float(pred[0]))

    #센터좌표값 리턴을 해줘야 합니다.
    add_safe = pm.get_safe_geo(center_label)
    safe_long = str(float(add_safe[0]))
    safe_lat = str(float(add_safe[1]))

    return jsonify({"answer": answer,
                    "safe_lat": safe_lat,
                    "safe_long": safe_long})
    #return "박성범 이건희"
    #return jsonify({"answer": pred})


@app.route('/predict', methods=['POST'])
# @app.route('/', methods=['GET'])

def home():

    # GEt 방식 변수받기
    # data1 = request.args.get('fire', type = int) # 삭제예정
    # data10 = request.args.get('emr', type = str) # 신고동이름 ex. 고덕동
    # data5 = request.args.get('rel', type = int) # 사건발생장소 ex. 1
    # data6 = request.args.get('stat', type = int) # 신고시각 ex. 12
    # data7 = request.args.get('jur', type = int) # 관할구역 ex. 1
    # data8 = request.args.get('x', type = float) # 위도 ex.38.6417
    # data9 = request.args.get('y', type = float) # 경도 ex.125.788109

    # 제이슨 데이터 받기
    jsonData = request.get_json()

    # # 제이슨 객체 내 요소들 변수에 받기
    # data1 = int(jsonData['fire'])
    data1 = 1
    data10 = str(jsonData['emr'])
    data5 = int(jsonData['rel'])
    data6 = int(jsonData['stat'])
    data7 = int(jsonData['jur'])
    data8 = float(jsonData['x'])
    data9 = float(jsonData['y'])

    print(data10)
    print(data5)
    print(data6)
    print(data7)
    print(data8)
    print(data9)


    if ((data8 == None) | (data9 == None)|(data10 == None)):
        return "좌표변수는 필수입니다."
    else:
        # dist_only : 현장과의 거리 계산
        data3 = pm.dist_only(data8, data9)
        # get_safe : 안전센터 레이블 반환
        data2 = pm.get_safe(data8, data9)
        # 동이름 반환
        data4 = pm.get_donginfo(data10)

    #안전센터 좌표
    add_safe = pm.get_safe_geo(data2)

    print("safe 좌표")
    print(add_safe)

    # 예측(모델 학습을 np.array로 시켜서 예측도 np.array로 진행함)
    arr = np.array([[data1, data2, data3, data4, data5, data6, data7]])
    pred = model.predict(arr)

    # 배열/튜플로 넘어오는 예측값을 json객체로 반환함
    answer = str(round(float(pred[0])))
    safe_long = str(float(add_safe[0]))
    safe_lat = str(float(add_safe[1]))

    print(answer)
    if ((data1 == None) | (data4 == None)| (data5 == None)|
            (data6 == None)| (data7 == None)):
        return "귀하가 보내주신 변수이름에 오류가 있습니다. 수정하세요."
    else :
        return jsonify({"answer": answer,
                        "safe_long" : safe_long,
                        "safe_lat" : safe_lat,
                        "pick_long" : data9,
                        "pick_lat" : data8})

if __name__ == "__main__":
    app.debug = True
    #app.run(port=80)
    app.run(host="192.168.2.147", port=80)


In [ ]:
from flask import Flask, render_template, redirect, url_for, request, jsonify
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import json
import pickle
import PSB_manage as pm

app = Flask(__name__)
model = pickle.load(open('aid_predict_time.pkl', 'rb')) # 모델 Input (XGBorlfhost)
center_model = pickle.load(open('aid_predict_dstn_ver2.pkl', 'rb')) # 소요거리 모델
cols_check = model.get_booster().feature_names
cols_check2 = center_model.get_booster().feature_names

@app.route('/predict2', methods=['POST'])
def centerPredict():
    # 센터, 시간대, 소요시간, 의식상태, 관할여부 받기
    # 모델에 넣고 거리 리턴
    jsonData = request.get_json()
    data1 = str(jsonData['safe_center'])
    data2 = int(jsonData['jurisd'])
    data3 = int(jsonData['statmt_tm'])
    data4 = float(jsonData['move_tm'])
    data5 = int(jsonData['conscs'])
    # json객체 출력 확인
    # 119안전센터 label 값 가져오기
    data6 = int(pm.get_centerlabel(data1))


    f0 = ((-9.99997093e-01*data6)+(8.44458518e-05*data3)+ (-1.40783964e-05*data2) + (-1.72248173e-05*data5) + (2.40958186e-03*data4))

    f1 = ((-1.14013325e-04 * data6) + (-9.99913736e-01 * data3) + (1.61452166e-03 * data2) + (-4.32766976e-03 * data5) + (-1.22951787e-02 * data4))

    f2 = ((2.40459382e-03 * data6) + (-1.25180261e-02 * data3) + (-2.47348604e-02 * data2) + (4.60709422e-02 * data5) + (9.98550535e-01 * data4))

    f3 = ((-1.28796553e-04 * data6) + (-3.75914134e-03 * data3) + (-2.33692511e-03 * data2) + (9.98922727e-01 * data5) + (-4.61928169e-02 * data4))

    f4 = ((-4.52960671e-05 * data6) + (-1.29636897e-03 * data3) + (-9.99690011e-01 * data2) + (-3.48203219e-03 * data5) + (-2.46185751e-02 * data4))


    # 소요시간 예측하기###########################################
    # 예측 데이터 담을 df 생성
    df = pd.DataFrame(columns=['f0', 'f1', 'f2',
                               'f3', 'f4'])
    # df에 행 추가
    df = df.append({'f0': f0, 'f1': f1, 'f2': f2,
                    'f3': f3, 'f4': f4}, ignore_index=True)
    # df 타입 변경
    # df2 = df.astype(np.int64)
    # df3 = df2.MOVE_TM.astype(np.float64)
    # df2 = df2.drop(columns='MOVE_TM')
    # df4 = pd.concat((df3, df2), axis=1)

    # df4 = df4[cols_check2]


    # 예측하기
    pred = center_model.predict(df)
    answer = str(round(float(pred[0]),2))

    #센터좌표값 리턴
    add_safe = pm.get_safe_geo(data6)
    safe_long = str(float(add_safe[0]))
    safe_lat = str(float(add_safe[1]))

    # 최종 값 리턴
    return jsonify({"answer": answer,
                    "safe_lat": safe_lat,
                    "safe_long": safe_long})


@app.route('/predict', methods=['POST'])

def home():
    # 제이슨 데이터 받기
    jsonData = request.get_json()

    # # 제이슨 객체 내 요소들 변수에 받기
    data1 = int(jsonData['conscs'])
    data10 = str(jsonData['address_emd'])
    data5 = int(jsonData['occurplc'])
    data6 = int(jsonData['statmnt_tm'])
    data7 = int(jsonData['jurisd'])
    data8 = float(jsonData['pick_lat'])
    data9 = float(jsonData['pick_long'])

    print(data10)
    print(data5)
    print(data6)
    print(data7)
    print(data8)
    print(data9)

    if ((data8 == None) | (data9 == None)|(data10 == None)):
        return "좌표변수는 필수입니다."
    else:
        # dist_only : 현장과의 거리 계산
        data3 = pm.dist_only(data8, data9)
        # get_safe : 안전센터 레이블 반환
        data2 = pm.get_safe(data8, data9)
        # 동이름 반환
        data4 = pm.get_donginfo(data10)




    #안전센터 좌표
    add_safe = pm.get_safe_geo(data2)

    print("safe 좌표")
    print(add_safe)

    df = pd.DataFrame(columns=['GOUT_SAFE_CENTER_NM', 'ONSPOT_DSTN',
       'EMRLF_EMD_NM', 'RELIF_OCCURPLC_TYPE', 'STATMNT_TM','JURISD_DIV_NM_CNT','CONSCS_STATE_TYPE' ])
    df = df.append({'CONSCS_STATE_TYPE': data1, 'GOUT_SAFE_CENTER_NM': data2, 'ONSPOT_DSTN': data3, 'EMRLF_EMD_NM': data4,
                    'RELIF_OCCURPLC_TYPE': data5, 'STATMNT_TM': data6, 'JURISD_DIV_NM_CNT': data7}, ignore_index=True)
    df2 = df.astype(np.int64)
    df3 = df2.ONSPOT_DSTN.astype(np.float64)
    df2 = df2.drop(columns='ONSPOT_DSTN')
    df4 = pd.concat((df3,df2), axis=1)
    df4 = df4[cols_check]

    # # 예측(모델 학습을 np.array로 시켜서 예측도 np.array로 진행함)
    # arr = np.array([[data1, data2, data3, data4, data5, data6, data7]])
    # pred = model.predict(arr)
    #
    # # 배열/튜플로 넘어오는 예측값을 json객체로 반환함
    # answer = str(round(float(pred[0])))

    # 예측하기
    pred = model.predict(df4)

    answer = str(round(float(pred[0])))

    safe_long = str(float(add_safe[0]))
    safe_lat = str(float(add_safe[1]))

    print(answer)
    if ((data1 == None) | (data4 == None)| (data5 == None)|
            (data6 == None)| (data7 == None)):
        return "귀하가 보내주신 변수이름에 오류가 있습니다. 수정하세요."
    else :
        return jsonify({"answer": answer,
                        "safe_long" : safe_long,
                        "safe_lat" : safe_lat,
                        "pick_long" : data9,
                        "pick_lat" : data8})

if __name__ == "__main__":
    app.debug = True
    #app.run(port=80)
    app.run(host="192.168.2.147", port=80)
